In [1]:
from utils.mongo_helper import MongoHelper
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
from tqdm import tqdm
import utils.time_util as tu

In [2]:
mongo = MongoHelper()
collection = mongo.db["data"]
data_df = pd.DataFrame(collection.find())
tqdm.pandas()

In [ ]:
def applied_delta(df):
    return tu.get_time_delta(
        tu.to_time_stamp(df['time']), 
        tu.to_time_stamp(df['fixtime'])
    )

data_df.loc[:,'day_of_fix'] = data_df.progress_apply(applied_delta, axis=1)

count = (data_df['day_of_fix'] <= 10).sum()
print(f"Fix in 10 days：{count}")

In [4]:
for index,row in data_df.iterrows():
    if row['day_of_fix'] < 0:
        print(row['_id'])

In [ ]:
grouped_df = data_df.groupby('tag').filter(lambda x: len(x)>1).groupby('tag')
grouped_df

In [ ]:
data = []
for name, group in grouped_df:
    data.append((name, group['day_of_fix']))
data

In [ ]:
def get_median(ele):
    return ele[1].median()

def get_mean(ele):
    return ele[1].mean()
    
data.sort(key=get_median)
data

In [ ]:
def get_labels(datas):
    new_labels = []
    for ele in datas:
        my_new_label = str(ele[0]).replace("_"," ")
        if my_new_label == "exception":
            my_new_label = "leak on exception"
        new_labels.append(my_new_label)
    return new_labels

plt.grid(False)
plt.boxplot([x[1] for x in data], 
            labels=get_labels(data), 
            vert=False, showfliers=False, showmeans=True)

# plt.xlabel("Fixtime")
# plt.ylabel("Type")
plt.savefig("../files/pics/type-fixtime-boxplot_no_filers.svg", dpi=300, format="svg", bbox_inches='tight')

In [ ]:
simplify_df = data_df[["tag", "day_of_fix"]]
simplify_df

In [ ]:

# plt.violinplot(simplify_df, vert=False, showmeans=True)


simplify_df['tag'] = simplify_df['tag'].str.replace('_',' ')

In [ ]:
my_order = simplify_df.groupby(by=["tag"])['day_of_fix'].max()
my_order = my_order.sort_values(ascending=False)
my_order.index

In [ ]:

# plt.xlabel("fix day")
# plt.ylabel("type")
# sns.violinplot(x='day_of_fix', y='tag', data=simplify_df, density_norm='width', cut=0, inner='point',
#                order=my_order.index)
# plt.savefig("../files/pics/type-fixtime-violin-plot.svg", dpi=300, format="svg", bbox_inches='tight')